<a href="https://colab.research.google.com/github/zhangxingeng/Capstone/blob/main/Capstone_ANOVA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
import pandas as pd
import numpy as np
!pip -q install statsmodels
import statsmodels

article = pd.read_csv("/content/gdrive/MyDrive/Capstone/cleaned_data/articles_clean.csv")
customer = pd.read_csv("/content/gdrive/MyDrive/Capstone/cleaned_data/customers_clean.csv")
trans = pd.read_csv("/content/gdrive/MyDrive/Capstone/cleaned_data/transactions_train.csv")

In [5]:
''' Replace string label to int label '''
catMap = {'LEFT CLUB':0, 'ACTIVE':2,'PRE-CREATE': 1, np.nan: 3}
newTable = customer.replace({ 'club_member_status': catMap})

from tabulate import tabulate
def drawTable(value: dict, header: list) -> str:
  ''' Draw a given dictionary into string '''
  return tabulate([(k,) + v for k, v in value.items()], headers=header, tablefmt='fancy_grid', numalign="right")

In [6]:
def univariateAnalysis(table:pd.DataFrame, numName:str, catName:str, catKeys: list)->None:
  valDict = {}
  num, cat = table[numName], table[catName]

  for k in catKeys:
    curr = num[cat==k]
    valDict[k] = (len(curr), curr.mean(), curr.var())

  header = ["key", "count", "mean", "variance"]
  print(drawTable(valDict, header))

''' Drawing Table '''
univariateAnalysis(newTable, 'age', 'club_member_status', [0, 1, 2, 3])


╒═══════╤═════════╤═════════╤════════════╕
│   key │   count │    mean │   variance │
╞═══════╪═════════╪═════════╪════════════╡
│     0 │     467 │ 34.0021 │    190.667 │
├───────┼─────────┼─────────┼────────────┤
│     1 │   92960 │ 40.4998 │    174.188 │
├───────┼─────────┼─────────┼────────────┤
│     2 │ 1272491 │ 36.0776 │    203.638 │
├───────┼─────────┼─────────┼────────────┤
│     3 │    6062 │ 37.4264 │    120.828 │
╘═══════╧═════════╧═════════╧════════════╛


In [7]:
from itertools import combinations
from statsmodels.stats.weightstats import ztest, ttest_ind

def statTests(table:pd.DataFrame, numName:str, catName:str, catKeys: list)->None:
  valDict = {}
  num, cat = table[numName], table[catName]
  for v1, v2 in list(combinations(catKeys,2)):
    r1, r2 = num[cat==v1], num[cat==v2]
    zvalue, pvalue1 = ztest(r1, r2, value=0)
    tvalue, pvalue2, dof = ttest_ind(r1, r2)
    valDict[f"{v1}, {v2}"] = (zvalue, tvalue, dof, pvalue1)
    if pvalue1-pvalue2 > 1e-10: print("Problem")

  header = ["key pair", "Z-Test", "T-Test", "DOF", "p-value" ]
  print(drawTable(valDict, header)) 

''' Drawing Table '''
statTests(newTable, 'age', 'club_member_status', [0, 1, 2, 3])

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


╒════════════╤══════════╤══════════╤═════════════╤═════════════╕
│ key pair   │   Z-Test │   T-Test │         DOF │     p-value │
╞════════════╪══════════╪══════════╪═════════════╪═════════════╡
│ 0, 1       │   -10.61 │   -10.61 │       93425 │ 2.67829e-26 │
├────────────┼──────────┼──────────┼─────────────┼─────────────┤
│ 0, 2       │ -3.14247 │ -3.14247 │ 1.27296e+06 │  0.00167531 │
├────────────┼──────────┼──────────┼─────────────┼─────────────┤
│ 0, 3       │ -6.35693 │ -6.35693 │        6527 │ 2.05823e-10 │
├────────────┼──────────┼──────────┼─────────────┼─────────────┤
│ 1, 2       │  91.6628 │  91.6628 │ 1.36545e+06 │           0 │
├────────────┼──────────┼──────────┼─────────────┼─────────────┤
│ 1, 3       │  17.7339 │  17.7339 │       99020 │ 2.29562e-70 │
├────────────┼──────────┼──────────┼─────────────┼─────────────┤
│ 2, 3       │ -7.34885 │ -7.34885 │ 1.27855e+06 │ 1.99916e-13 │
╘════════════╧══════════╧══════════╧═════════════╧═════════════╛


In [8]:
from scipy import stats
def anovaTable(table: pd.DataFrame, numName: str, catName: str, keys: list) -> None: # comment out for testing
# def anovaTable(): # comment in for testing
  sumList, sum2List, cntList =[], [], []
  # keys = [0, 1, 2] # comment in for testing
  # num = {0: [86,65,90,75], 1:[96,80,70,80,91], 2: [85,90,95,70,70]} # comment in for testing
  for k in keys:
    num, cat = table[numName], table[catName] # comment out for testing
    curr = num[cat==k] # comment out for testing
    # curr = np.array(num[k]) # comment in for testing
    sumList.append(curr.sum())
    sum2List.append(np.square(curr).sum())
    cntList.append(len(curr))

  sum_sqsum_over_c = sum([s2/c for s2, c in zip(np.square(sumList), cntList)])
  sumsq_over_sum_c = sum(sumList)**2/sum(cntList)
  sum_sum_sq = sum(sum2List)
  ssb = sum_sqsum_over_c - sumsq_over_sum_c
  ssw = sum_sum_sq - sum_sqsum_over_c
  sst = sum_sum_sq - sumsq_over_sum_c

  dfw = sum(cntList) - float(len(cntList))
  dfb = len(cntList) - 1.0
  dft = sum(cntList) - 1.0

  msb = ssb/dfb
  msw = ssw/dfw
  f = msb/msw
  pf = 1-stats.f.cdf(f, dfb, dfw)

  ''' Print a Table For the Result '''
  data = {"Between Groups":(ssb, dfb, round(msb, 3), round(f, 3), round(pf, 3)), 
          "Within Groups":(ssw, dfw, round(msw, 3), "", ""),
          "Total":(sst, dft, "", "", "")}
  header = ["Source of Variation", "Sum of Squares", "Degree of freedom", "Meansquare", "F-value", "probability"]
  print(drawTable(data, header))

''' Drawing Table '''
anovaTable(newTable, 'age', 'club_member_status', [0,1,2])

╒═══════════════════════╤══════════════════╤═════════════════════╤══════════════╤═══════════╤═══════════════╕
│ Source of Variation   │   Sum of Squares │   Degree of freedom │ Meansquare   │ F-value   │ probability   │
╞═══════════════════════╪══════════════════╪═════════════════════╪══════════════╪═══════════╪═══════════════╡
│ Between Groups        │      1.69677e+06 │                   2 │ 848383.557   │ 4207.648  │ 0.0           │
├───────────────────────┼──────────────────┼─────────────────────┼──────────────┼───────────┼───────────────┤
│ Within Groups         │      2.75408e+08 │         1.36592e+06 │ 201.629      │           │               │
├───────────────────────┼──────────────────┼─────────────────────┼──────────────┼───────────┼───────────────┤
│ Total                 │      2.77105e+08 │         1.36592e+06 │              │           │               │
╘═══════════════════════╧══════════════════╧═════════════════════╧══════════════╧═══════════╧═══════════════╛


In [9]:
color = article[["article_id", "colour_group_name"]]
price = trans[["article_id","price"]]
priceColor = pd.merge(price, color, how="inner", on="article_id")
print(priceColor.head())

   article_id     price colour_group_name
0   663713001  0.050831             Black
1   663713001  0.049475             Black
2   663713001  0.050831             Black
3   663713001  0.050831             Black
4   663713001  0.050831             Black


In [10]:
univariateAnalysis(priceColor, 'price', 'colour_group_name', ["Black", "Dark Blue", "White"])
statTests(priceColor, 'price', 'colour_group_name', ["Black", "Dark Blue", "White"])
anovaTable(priceColor, 'price', 'colour_group_name', ["Black", "Dark Blue", "White"])

╒═══════════╤══════════╤═══════════╤═════════════╕
│ key       │    count │      mean │    variance │
╞═══════════╪══════════╪═══════════╪═════════════╡
│ Black     │ 11036956 │ 0.0280246 │ 0.000356485 │
├───────────┼──────────┼───────────┼─────────────┤
│ Dark Blue │  2180620 │ 0.0293807 │ 0.000418592 │
├───────────┼──────────┼───────────┼─────────────┤
│ White     │  3368276 │ 0.0226262 │ 0.000211745 │
╘═══════════╧══════════╧═══════════╧═════════════╛
╒══════════════════╤══════════╤══════════╤═════════════╤═══════════╕
│ key pair         │   Z-Test │   T-Test │         DOF │   p-value │
╞══════════════════╪══════════╪══════════╪═════════════╪═══════════╡
│ Black, Dark Blue │ -95.5516 │ -95.5516 │ 1.32176e+07 │         0 │
├──────────────────┼──────────┼──────────┼─────────────┼───────────┤
│ Black, White     │  482.809 │  482.809 │ 1.44052e+07 │         0 │
├──────────────────┼──────────┼──────────┼─────────────┼───────────┤
│ Dark Blue, White │  453.967 │  453.967 │ 5.54889e+06 │  

In [15]:
h = ["Name", "Keys", "Columns", "Rows", "Size"]
d = {
"articles.csv": ("Article_id", 25, 105542, "35MB"), 
"customers.csv": ("customer_id", 7, 1356119, "207MB"), 
"transaction_train.csv": ("Article_id, customer_id", 5, 31788324, "3.49GB")
}
print(drawTable(d, h))

╒═══════════════════════╤═════════════════════════╤═══════════╤══════════╤════════╕
│ Name                  │ Keys                    │   Columns │     Rows │ Size   │
╞═══════════════════════╪═════════════════════════╪═══════════╪══════════╪════════╡
│ articles.csv          │ Article_id              │        25 │   105542 │ 35MB   │
├───────────────────────┼─────────────────────────┼───────────┼──────────┼────────┤
│ customers.csv         │ customer_id             │         7 │  1356119 │ 207MB  │
├───────────────────────┼─────────────────────────┼───────────┼──────────┼────────┤
│ transaction_train.csv │ Article_id, customer_id │         5 │ 31788324 │ 3.49GB │
╘═══════════════════════╧═════════════════════════╧═══════════╧══════════╧════════╛
